In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

subscription_id = "<subscription_id>"
resource_group = "<resource_group_name>"
workspace = "<workspace_name>"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace,
)

In [ ]:
cpu_compute_target = "cpu-cluster"
cpu_cluster = ml_client.compute.get(cpu_compute_target)

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

train_data_name = 'aml-sdk-handson-train-data'

train_data = Data(
    path='../data/nyc_taxi_dataset_train.csv',
    type=AssetTypes.URI_FILE,
    description="SDK v2 ハンズオンのための学習用データアセット",
    name=train_data_name,
)

train_data_asset = ml_client.data.create_or_update(train_data)

valid_data_name = 'aml-sdk-handson-valid-data'

valid_data = Data(
    path='../data/nyc_taxi_dataset_valid.csv',
    type=AssetTypes.URI_FILE,
    description="SDK v2 ハンズオンのための検証用データアセット",
    name=valid_data_name,
)

valid_data_asset = ml_client.data.create_or_update(valid_data)

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-sdk-handson-sklearn"

env = Environment(
    name=custom_env_name,
    description="SDK v2 ハンズオンのための Environment",
    conda_file="../conda_env.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

environment_asset = ml_client.environments.create_or_update(env)

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.entities import CommandJobLimits

job = command(
    inputs={
        "train_data": Input(type=AssetTypes.URI_FILE, path=f"azureml:{train_data_name}:{train_data_asset.version}"),
        "valid_data": Input(type=AssetTypes.URI_FILE, path=f"azureml:{valid_data_name}:{valid_data_asset.version}"),
    },
    code="../script/",
    command="python build_model.py --input_train_data ${{inputs.train_data}} --input_valid_data ${{inputs.valid_data}}",
    environment=f"{custom_env_name}:{environment_asset.version}",
    compute=cpu_compute_target,
    experiment_name="aml-sdk-handson-sklearn-job",
    limits=CommandJobLimits(timeout=300)
)

In [ ]:
job = ml_client.create_or_update(job)
ml_client.jobs.stream(job.name)